In [1]:
cd ~/code/git/pudl/pudl

/Users/zaneselvans/code/git/pudl/pudl


In [2]:
%load_ext autoreload
%autoreload 1
%aimport pudl
%aimport ferc1

In [14]:
import pandas as pd
import pudl
import ferc1

In [15]:
pudl.ferc1.f1_slurp()
f1_engine = create_engine('postgresql://catalyst@localhost:5432/ferc_f1')

In [6]:
f1_respondents = pd.read_sql('''SELECT DISTINCT respondent_id, respondent_name from f1_respondent_id''', f1_engine)
f1_plants = pd.read_sql('''SELECT DISTINCT r.respondent_id, r.respondent_name, f.plant_name
                           FROM f1_respondent_id r JOIN f1_fuel f on f.respondent_id = r.respondent_id
                           WHERE f.plant_name <> '' ''',f1_engine)
f1_fuel = pd.read_sql('''SELECT * from f1_fuel''', f1_engine)
f1_respondents.to_csv('../results/id_mapping/ferc1_inputs/f1_respondents.csv')
f1_plants.to_csv('../results/id_mapping/ferc1_inputs/f1_plants.csv')

In [11]:
small_plants = pd.read_sql('''SELECT DISTINCT r.respondent_id, r.respondent_name, p.plant_name, p.kind_of_fuel, p.capacity_rating
                            FROM f1_respondent_id r JOIN f1_gnrt_plant p ON r.respondent_id = p.respondent_id
                            WHERE p.capacity_rating > 5.0''', f1_engine)
small_plants.to_csv('../results/id_mapping/ferc1_inputs/f1_gnrt_plant.csv')

In [12]:
hydro_plants = pd.read_sql('''SELECT DISTINCT r.respondent_id, r.respondent_name, p.plant_name, p.plant_kind,
                                              p.tot_capacity
                              FROM f1_respondent_id r JOIN f1_hydro p ON r.respondent_id = p.respondent_id
                              WHERE p.tot_capacity > 5.0''', f1_engine)
hydro_plants.to_csv('../results/id_mapping/ferc1_inputs/f1_hydro.csv')

In [13]:
pumped_storage = pd.read_sql('''SELECT DISTINCT r.respondent_id, r.respondent_name, p.plant_name, p.plant_kind,
                                              p.tot_capacity
                              FROM f1_respondent_id r JOIN f1_pumped_storage p ON r.respondent_id = p.respondent_id
                              WHERE p.tot_capacity > 5.0''', f1_engine)
pumped_storage.to_csv('../results/id_mapping/ferc1_inputs/f1_pumped_storage.csv')